# 🛠️ Feature Engineering y Regresión Polinomial

## 📚 Objetivos de Aprendizaje
En este notebook aprenderás:
- **Qué es** feature engineering y por qué es crucial
- **Cómo crear** features polinomiales para capturar relaciones no lineales
- **Cuándo usar** regresión polinomial vs lineal
- **Cómo evitar** overfitting y underfitting
- **Técnicas avanzadas** de creación de features

## 🎯 Problema Motivador
**Limitación de regresión lineal**: Solo puede modelar relaciones lineales

**¿Qué pasa cuando los datos tienen patrones no lineales?**
- Relaciones cuadráticas: y = x²
- Relaciones cúbicas: y = x³ 
- Interacciones entre features: y = x₁ × x₂
- Patrones sinusoidales: y = sin(x)

**Solución**: Feature Engineering + Regresión Polinomial

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
from mpl_toolkits.mplot3d import Axes3D

# Configuración
plt.style.use('default')
np.set_printoptions(precision=2, suppress=True)
plt.rcParams['figure.figsize'] = (12, 8)

print("Librerías importadas correctamente")
print("Listo para feature engineering")

## 1. 📊 Motivación: Limitaciones de la Regresión Lineal

### 1.1 Datos con Patrones No Lineales

In [ ]:
# Crear datos con diferentes patrones no lineales
def generar_datos_no_lineales():
    """Genera datasets con diferentes patrones no lineales"""
    np.random.seed(42)
    
    # Dataset 1: Relación cuadrática
    x1 = np.linspace(0, 4, 20)
    y1 = 1 + x1**2 + np.random.normal(0, 2, len(x1))
    
    # Dataset 2: Relación cúbica
    x2 = np.linspace(-2, 2, 25)
    y2 = x2**3 - x2**2 + 2*x2 + 5 + np.random.normal(0, 1, len(x2))
    
    # Dataset 3: Relación sinusoidal
    x3 = np.linspace(0, 4*np.pi, 30)
    y3 = 2 + np.sin(x3) + 0.5*x3 + np.random.normal(0, 0.3, len(x3))
    
    # Dataset 4: Relación exponencial
    x4 = np.linspace(0, 3, 18)
    y4 = 2 * np.exp(0.5 * x4) + np.random.normal(0, 1, len(x4))
    
    return (x1, y1), (x2, y2), (x3, y3), (x4, y4)

# Funciones auxiliares de regresión lineal (del notebook anterior)
def zscore_normalize_features(X):
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    X_norm = (X - mu) / sigma
    return X_norm, mu, sigma

def calcular_costo(X, y, w, b):
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    m = X.shape[0]
    predictions = X @ w + b
    cost = np.sum((predictions - y)**2) / (2 * m)
    return cost

def gradient_descent_simple(X, y, w_init, b_init, alpha, iterations):
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    m, n = X.shape
    w = copy.deepcopy(w_init)
    b = b_init
    
    for i in range(iterations):
        predictions = X @ w + b
        errors = predictions - y
        
        dw = (X.T @ errors) / m
        db = np.sum(errors) / m
        
        w = w - alpha * dw
        b = b - alpha * db
    
    return w, b

# Generar datasets
datasets = generar_datos_no_lineales()
nombres = ['Cuadrática (y = x²)', 'Cúbica (y = x³)', 'Sinusoidal (y = sin(x))', 'Exponencial (y = eˣ)']

# Visualizar datasets y ajustes lineales
plt.figure(figsize=(16, 12))

for i, ((x, y), nombre) in enumerate(zip(datasets, nombres)):
    plt.subplot(2, 2, i+1)
    
    # Datos originales
    plt.scatter(x, y, alpha=0.7, s=50, color='red', label='Datos reales')
    
    # Ajuste lineal
    x_norm, mu, sigma = zscore_normalize_features(x)
    w_linear, b_linear = gradient_descent_simple(x_norm, y, np.array([0.]), 0., 0.1, 1000)
    
    # Predicciones lineales
    x_plot = np.linspace(np.min(x), np.max(x), 100)
    x_plot_norm = (x_plot - mu) / sigma
    y_linear = x_plot_norm * w_linear[0] + b_linear
    
    plt.plot(x_plot, y_linear, 'b-', linewidth=2, label='Ajuste Lineal')
    
    # Calcular R² para el ajuste lineal
    y_pred_linear = x_norm.flatten() * w_linear[0] + b_linear
    ss_res = np.sum((y - y_pred_linear)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r2_linear = 1 - (ss_res / ss_tot)
    
    plt.title(f'{nombre}\nR² Lineal = {r2_linear:.3f}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observaciones:")
print("  • La regresión lineal falla en capturar patrones no lineales")
print("  • R² bajo indica mal ajuste para datos curvos")
print("  • Necesitamos una forma de modelar relaciones no lineales")
print("\nSolución: Feature Engineering + Regresión Polinomial")

## 2. 🎨 Conceptos de Feature Engineering

### 2.1 ¿Qué es Feature Engineering?

**Definición**: Proceso de crear nuevas features a partir de las existentes para mejorar el rendimiento del modelo.

**Idea clave**: Aunque usamos regresión *lineal*, podemos modelar relaciones *no lineales* creando features no lineales.

**Ejemplo**: 
- Feature original: x
- Features engineered: x², x³, √x, log(x), sin(x)
- Modelo: y = w₁x + w₂x² + w₃x³ + b

In [ ]:
# Demostración conceptual de feature engineering
def crear_features_polinomiales(x, grado):
    """
    Crea features polinomiales hasta el grado especificado
    
    Args:
        x (array): Feature original
        grado (int): Grado máximo del polinomio
        
    Returns:
        X_poly (array): Matriz con features polinomiales
    """
    if x.ndim == 1:
        x = x.reshape(-1, 1)
    
    m = x.shape[0]
    X_poly = np.ones((m, grado + 1))  # Incluir término constante
    
    for i in range(1, grado + 1):
        X_poly[:, i] = (x.flatten()) ** i
    
    return X_poly[:, 1:]  # Excluir término constante (lo maneja b)

# Ejemplo con el primer dataset (cuadrático)
x_cuad, y_cuad = datasets[0]

print("Ejemplo: Transformación de Features")
print("=" * 40)

# Features originales
print(f"Feature original (x):")
print(f"Primeros 5 valores: {x_cuad[:5]}")
print(f"Shape: {x_cuad.shape}")

# Features polinomiales de grado 2
X_poly_2 = crear_features_polinomiales(x_cuad, 2)
print(f"\nFeatures polinomiales (grado 2):")
print(f"Columnas: [x, x²]")
print(f"Primeras 5 filas:")
print(X_poly_2[:5])
print(f"Shape: {X_poly_2.shape}")

# Features polinomiales de grado 3
X_poly_3 = crear_features_polinomiales(x_cuad, 3)
print(f"\nFeatures polinomiales (grado 3):")
print(f"Columnas: [x, x², x³]")
print(f"Primeras 5 filas:")
print(X_poly_3[:5])
print(f"Shape: {X_poly_3.shape}")

print(f"\nObservación clave:")
print(f"• Transformamos 1 feature original en múltiples features")
print(f"• El modelo sigue siendo 'lineal' en los parámetros w")
print(f"• Pero puede capturar relaciones no lineales en x")

### 2.2 Regresión Polinomial en Acción

In [ ]:
# Comparar diferentes grados polinomiales
def ajustar_polinomio(x, y, grado):
    """
    Ajusta un polinomio de grado especificado a los datos
    """
    # Crear features polinomiales
    X_poly = crear_features_polinomiales(x, grado)
    
    # Normalizar features
    X_norm, mu, sigma = zscore_normalize_features(X_poly)
    
    # Entrenar modelo
    w_init = np.zeros(X_norm.shape[1])
    w, b = gradient_descent_simple(X_norm, y, w_init, 0., 0.1, 1000)
    
    return w, b, mu, sigma

def predecir_polinomio(x_new, w, b, mu, sigma, grado):
    """
    Hace predicciones con modelo polinomial
    """
    X_poly = crear_features_polinomiales(x_new, grado)
    X_norm = (X_poly - mu) / sigma
    return X_norm @ w + b

# Probar diferentes grados en el dataset cuadrático
grados = [1, 2, 3, 5]
colores = ['blue', 'green', 'red', 'purple']

plt.figure(figsize=(15, 10))

# Datos originales
plt.scatter(x_cuad, y_cuad, alpha=0.7, s=50, color='black', 
           label='Datos reales', zorder=5)

# Ajustar y visualizar cada grado
x_plot = np.linspace(np.min(x_cuad), np.max(x_cuad), 100)
resultados = []

for grado, color in zip(grados, colores):
    # Ajustar modelo
    w, b, mu, sigma = ajustar_polinomio(x_cuad, y_cuad, grado)
    
    # Predicciones
    y_plot = predecir_polinomio(x_plot, w, b, mu, sigma, grado)
    y_pred = predecir_polinomio(x_cuad, w, b, mu, sigma, grado)
    
    # Calcular R²
    ss_res = np.sum((y_cuad - y_pred)**2)
    ss_tot = np.sum((y_cuad - np.mean(y_cuad))**2)
    r2 = 1 - (ss_res / ss_tot)
    
    # Visualizar
    plt.plot(x_plot, y_plot, color=color, linewidth=2, 
             label=f'Grado {grado} (R² = {r2:.3f})')
    
    resultados.append({
        'grado': grado,
        'r2': r2,
        'w': w,
        'b': b
    })

plt.title('Comparación de Grados Polinomiales\nDatos Cuadráticos Reales')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Mostrar resultados tabulados
print("Resultados por grado polinomial:")
print("-" * 30)
print(f"{'Grado':<6} {'R²':<8} {'Interpretación':<20}")
print("-" * 35)

for resultado in resultados:
    grado = resultado['grado']
    r2 = resultado['r2']
    
    if r2 < 0.7:
        interp = "Underfitting"
    elif r2 > 0.95 and grado > 3:
        interp = "Posible Overfitting"
    else:
        interp = "Buen ajuste"
    
    print(f"{grado:<6} {r2:<8.3f} {interp:<20}")

print(f"\nObservaciones:")
print(f"  • Grado 1 (lineal): Underfitting claro")
print(f"  • Grado 2: Perfecto para datos cuadráticos")
print(f"  • Grados altos: Mejor R² pero riesgo de overfitting")

## 3. 🔍 Exploración Sistemática de Grados Polinomiales

In [ ]:
# Análisis sistemático de todos los datasets
def analizar_grados_sistematico(datasets, nombres, grados_prueba):
    """
    Analiza diferentes grados polinomiales para múltiples datasets
    """
    plt.figure(figsize=(20, 15))
    
    resultados_completos = []
    
    for dataset_idx, ((x, y), nombre_dataset) in enumerate(zip(datasets, nombres)):
        # Subplot para este dataset
        plt.subplot(2, 2, dataset_idx + 1)
        
        # Datos originales
        plt.scatter(x, y, alpha=0.7, s=30, color='black', label='Datos reales')
        
        colores = plt.cm.viridis(np.linspace(0, 1, len(grados_prueba)))
        x_plot = np.linspace(np.min(x), np.max(x), 100)
        
        dataset_resultados = []
        
        for grado, color in zip(grados_prueba, colores):
            try:
                # Ajustar modelo
                w, b, mu, sigma = ajustar_polinomio(x, y, grado)
                
                # Predicciones
                y_pred = predecir_polinomio(x, w, b, mu, sigma, grado)
                y_plot = predecir_polinomio(x_plot, w, b, mu, sigma, grado)
                
                # Métricas
                ss_res = np.sum((y - y_pred)**2)
                ss_tot = np.sum((y - np.mean(y))**2)
                r2 = 1 - (ss_res / ss_tot)
                
                mse = np.mean((y - y_pred)**2)
                
                # Solo mostrar algunos grados para claridad visual
                if grado in [1, 2, 3, 6]:
                    plt.plot(x_plot, y_plot, color=color, linewidth=2, 
                            label=f'Grado {grado} (R²={r2:.2f})')
                
                dataset_resultados.append({
                    'grado': grado,
                    'r2': r2,
                    'mse': mse,
                    'dataset': nombre_dataset
                })
                
            except Exception as e:
                print(f"Error con grado {grado} en {nombre_dataset}: {e}")
        
        resultados_completos.extend(dataset_resultados)
        
        plt.title(f'{nombre_dataset}', fontsize=12)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.legend(fontsize=8)
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return resultados_completos

# Ejecutar análisis sistemático
grados_prueba = range(1, 11)  # Grados 1 a 10
resultados_completos = analizar_grados_sistematico(datasets, nombres, grados_prueba)

print("Análisis sistemático completado")
print(f"Total de experimentos: {len(resultados_completos)}")

### 3.1 Análisis de Resultados

In [ ]:
# Visualizar tendencias de R² vs grado
plt.figure(figsize=(15, 10))

# Organizar datos por dataset
datasets_resultados = {}
for resultado in resultados_completos:
    dataset = resultado['dataset']
    if dataset not in datasets_resultados:
        datasets_resultados[dataset] = {'grados': [], 'r2': [], 'mse': []}
    datasets_resultados[dataset]['grados'].append(resultado['grado'])
    datasets_resultados[dataset]['r2'].append(resultado['r2'])
    datasets_resultados[dataset]['mse'].append(resultado['mse'])

# Subplot 1: R² vs Grado
plt.subplot(2, 2, 1)
for dataset, datos in datasets_resultados.items():
    plt.plot(datos['grados'], datos['r2'], 'o-', linewidth=2, markersize=6, 
             label=dataset.split('(')[0].strip())

plt.title('R² vs Grado Polinomial')
plt.xlabel('Grado Polinomial')
plt.ylabel('R² Score')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 1.05)

# Subplot 2: MSE vs Grado
plt.subplot(2, 2, 2)
for dataset, datos in datasets_resultados.items():
    plt.plot(datos['grados'], datos['mse'], 'o-', linewidth=2, markersize=6, 
             label=dataset.split('(')[0].strip())

plt.title('MSE vs Grado Polinomial')
plt.xlabel('Grado Polinomial')
plt.ylabel('MSE')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Subplot 3: Mejor grado por dataset
plt.subplot(2, 2, 3)
mejores_grados = []
nombres_datasets = []
mejores_r2 = []

for dataset, datos in datasets_resultados.items():
    # Encontrar grado con mejor R² (pero penalizar grados muy altos)
    r2_array = np.array(datos['r2'])
    grados_array = np.array(datos['grados'])
    
    # Penalizar grados altos (regularización simple)
    penalizacion = 0.01 * grados_array
    r2_penalizado = r2_array - penalizacion
    
    mejor_idx = np.argmax(r2_penalizado)
    mejores_grados.append(grados_array[mejor_idx])
    nombres_datasets.append(dataset.split('(')[0].strip())
    mejores_r2.append(r2_array[mejor_idx])

bars = plt.bar(nombres_datasets, mejores_grados, 
               color=['skyblue', 'lightgreen', 'lightcoral', 'lightsalmon'],
               alpha=0.7, edgecolor='black')

plt.title('Mejor Grado por Dataset\n(con penalización por complejidad)')
plt.xlabel('Dataset')
plt.ylabel('Grado Óptimo')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')

# Añadir valores en las barras
for bar, grado, r2 in zip(bars, mejores_grados, mejores_r2):
    plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
             f'{grado}\n(R²={r2:.2f})', ha='center', va='bottom')

# Subplot 4: Distribución de complejidad
plt.subplot(2, 2, 4)
todos_grados = [resultado['grado'] for resultado in resultados_completos]
todos_r2 = [resultado['r2'] for resultado in resultados_completos]

plt.scatter(todos_grados, todos_r2, alpha=0.6, s=50)
plt.title('Complejidad vs Rendimiento\n(Todos los experimentos)')
plt.xlabel('Grado Polinomial')
plt.ylabel('R² Score')
plt.grid(True, alpha=0.3)

# Línea de tendencia
z = np.polyfit(todos_grados, todos_r2, 2)
p = np.poly1d(z)
x_trend = np.linspace(1, 10, 100)
plt.plot(x_trend, p(x_trend), 'r--', alpha=0.8, linewidth=2)

plt.tight_layout()
plt.show()

# Análisis estadístico
print("Análisis de Grados Óptimos:")
print("=" * 30)
for dataset, grado, r2 in zip(nombres_datasets, mejores_grados, mejores_r2):
    print(f"{dataset:<12}: Grado {grado} (R² = {r2:.3f})")

print(f"\nPatrones observados:")
print(f"  • Datos cuadráticos: Mejor con grado 2-3")
print(f"  • Datos cúbicos: Mejor con grado 3-4")
print(f"  • Datos complejos: Requieren grados más altos")
print(f"  • Overfitting: Grados muy altos (>7) no siempre mejoran")

## 4. ⚠️ Overfitting vs Underfitting

### 4.1 Demostración del Problema

In [ ]:
# Crear dataset con poco ruido para demostrar overfitting claramente
np.random.seed(123)
x_demo = np.linspace(0, 1, 15)
y_demo = 4*x_demo**2 - 2*x_demo + 1 + np.random.normal(0, 0.1, len(x_demo))

# Probar grados extremos
grados_extremos = [1, 2, 8, 12]
nombres_extremos = ['Underfitting (Grado 1)', 'Apropiado (Grado 2)', 
                   'Overfitting (Grado 8)', 'Overfitting Severo (Grado 12)']

plt.figure(figsize=(16, 12))

for i, (grado, nombre) in enumerate(zip(grados_extremos, nombres_extremos)):
    plt.subplot(2, 2, i+1)
    
    # Datos de entrenamiento
    plt.scatter(x_demo, y_demo, alpha=0.8, s=60, color='red', 
               label='Datos entrenamiento', zorder=5)
    
    # Ajustar modelo
    w, b, mu, sigma = ajustar_polinomio(x_demo, y_demo, grado)
    
    # Predicciones en datos de entrenamiento
    y_pred_train = predecir_polinomio(x_demo, w, b, mu, sigma, grado)
    
    # Calcular R² en entrenamiento
    r2_train = 1 - np.sum((y_demo - y_pred_train)**2) / np.sum((y_demo - np.mean(y_demo))**2)
    
    # Línea suave para visualización
    x_plot = np.linspace(0, 1, 100)
    y_plot = predecir_polinomio(x_plot, w, b, mu, sigma, grado)
    
    plt.plot(x_plot, y_plot, 'b-', linewidth=2, label='Modelo')
    
    # Función verdadera (para referencia)
    y_true = 4*x_plot**2 - 2*x_plot + 1
    plt.plot(x_plot, y_true, 'g--', linewidth=2, alpha=0.7, label='Función verdadera')
    
    plt.title(f'{nombre}\nR² entrenamiento = {r2_train:.3f}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(-1, 3)

plt.tight_layout()
plt.show()

print("Características de cada caso:")
print("-" * 40)
print("1. Underfitting (Grado 1):")
print("   • Modelo demasiado simple")
print("   • No captura la curvatura real")
print("   • Bias alto, varianza baja")

print("\n2. Ajuste Apropiado (Grado 2):")
print("   • Complejidad correcta para los datos")
print("   • Captura el patrón sin sobreajustar")
print("   • Balance óptimo bias-varianza")

print("\n3. Overfitting (Grados 8+):")
print("   • Modelo demasiado complejo")
print("   • Memoriza ruido en datos de entrenamiento")
print("   • Bias bajo, varianza alta")
print("   • Mal rendimiento en datos nuevos")

### 4.2 Simulación de Train/Test Split

In [ ]:
# Simular división train/test para demostrar overfitting
def simular_train_test_split():
    """
    Crea datos de entrenamiento y prueba para demostrar overfitting
    """
    np.random.seed(42)
    
    # Función verdadera: cuadrática con ruido
    def funcion_verdadera(x):
        return 2 + x - 0.5*x**2 + 0.1*x**3
    
    # Datos de entrenamiento (pocos puntos)
    x_train = np.linspace(0, 2, 12)
    y_train = funcion_verdadera(x_train) + np.random.normal(0, 0.15, len(x_train))
    
    # Datos de prueba (más puntos, sin ruido para ver patrón real)
    x_test = np.linspace(0, 2, 20)
    y_test = funcion_verdadera(x_test) + np.random.normal(0, 0.1, len(x_test))
    
    return x_train, y_train, x_test, y_test, funcion_verdadera

# Generar datos
x_train, y_train, x_test, y_test, func_verdadera = simular_train_test_split()

# Evaluar diferentes grados
grados_evaluar = range(1, 10)
resultados_train_test = []

for grado in grados_evaluar:
    # Entrenar en datos de entrenamiento
    w, b, mu, sigma = ajustar_polinomio(x_train, y_train, grado)
    
    # Evaluar en entrenamiento
    y_pred_train = predecir_polinomio(x_train, w, b, mu, sigma, grado)
    mse_train = np.mean((y_train - y_pred_train)**2)
    r2_train = 1 - np.sum((y_train - y_pred_train)**2) / np.sum((y_train - np.mean(y_train))**2)
    
    # Evaluar en prueba
    y_pred_test = predecir_polinomio(x_test, w, b, mu, sigma, grado)
    mse_test = np.mean((y_test - y_pred_test)**2)
    r2_test = 1 - np.sum((y_test - y_pred_test)**2) / np.sum((y_test - np.mean(y_test))**2)
    
    resultados_train_test.append({
        'grado': grado,
        'mse_train': mse_train,
        'mse_test': mse_test,
        'r2_train': r2_train,
        'r2_test': r2_test,
        'gap': mse_test - mse_train
    })

# Visualizar curvas de aprendizaje
plt.figure(figsize=(15, 10))

# Subplot 1: MSE Train vs Test
plt.subplot(2, 2, 1)
grados = [r['grado'] for r in resultados_train_test]
mse_train = [r['mse_train'] for r in resultados_train_test]
mse_test = [r['mse_test'] for r in resultados_train_test]

plt.plot(grados, mse_train, 'o-', color='blue', linewidth=2, 
         markersize=6, label='MSE Entrenamiento')
plt.plot(grados, mse_test, 'o-', color='red', linewidth=2, 
         markersize=6, label='MSE Prueba')

plt.title('Curvas de Aprendizaje: MSE')
plt.xlabel('Grado Polinomial')
plt.ylabel('MSE')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Marcar punto de overfitting
# Encontrar donde MSE test empieza a subir
min_mse_test_idx = np.argmin(mse_test)
mejor_grado = grados[min_mse_test_idx]
plt.axvline(x=mejor_grado, color='green', linestyle='--', alpha=0.7, 
           label=f'Óptimo (Grado {mejor_grado})')
plt.legend()

# Subplot 2: Gap Train-Test
plt.subplot(2, 2, 2)
gaps = [r['gap'] for r in resultados_train_test]
plt.plot(grados, gaps, 'o-', color='purple', linewidth=2, markersize=6)
plt.title('Gap: MSE Test - MSE Train\n(Indicador de Overfitting)')
plt.xlabel('Grado Polinomial')
plt.ylabel('Gap MSE')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)

# Subplot 3: Mejor modelo (grado óptimo)
plt.subplot(2, 2, 3)
w_opt, b_opt, mu_opt, sigma_opt = ajustar_polinomio(x_train, y_train, mejor_grado)

plt.scatter(x_train, y_train, color='blue', s=60, label='Train', alpha=0.8)
plt.scatter(x_test, y_test, color='red', s=60, label='Test', alpha=0.8)

x_plot = np.linspace(0, 2, 100)
y_plot_opt = predecir_polinomio(x_plot, w_opt, b_opt, mu_opt, sigma_opt, mejor_grado)
y_verdadera = func_verdadera(x_plot)

plt.plot(x_plot, y_plot_opt, 'g-', linewidth=2, label=f'Modelo (Grado {mejor_grado})')
plt.plot(x_plot, y_verdadera, 'k--', linewidth=2, alpha=0.7, label='Función Verdadera')

plt.title(f'Modelo Óptimo (Grado {mejor_grado})')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 4: Modelo con overfitting
plt.subplot(2, 2, 4)
grado_over = 8  # Grado alto para mostrar overfitting
w_over, b_over, mu_over, sigma_over = ajustar_polinomio(x_train, y_train, grado_over)

plt.scatter(x_train, y_train, color='blue', s=60, label='Train', alpha=0.8)
plt.scatter(x_test, y_test, color='red', s=60, label='Test', alpha=0.8)

y_plot_over = predecir_polinomio(x_plot, w_over, b_over, mu_over, sigma_over, grado_over)
plt.plot(x_plot, y_plot_over, 'r-', linewidth=2, label=f'Modelo (Grado {grado_over})')
plt.plot(x_plot, y_verdadera, 'k--', linewidth=2, alpha=0.7, label='Función Verdadera')

plt.title(f'Overfitting (Grado {grado_over})')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tabla de resultados
print("Análisis Train/Test Split:")
print("-" * 50)
print(f"{'Grado':<6} {'MSE Train':<10} {'MSE Test':<10} {'Gap':<8} {'Status':<15}")
print("-" * 50)

for resultado in resultados_train_test:
    grado = resultado['grado']
    mse_tr = resultado['mse_train']
    mse_te = resultado['mse_test']
    gap = resultado['gap']
    
    if grado == mejor_grado:
        status = "ÓPTIMO"
    elif gap < 0.01:
        status = "Underfitting"
    elif gap > 0.05:
        status = "Overfitting"
    else:
        status = "Aceptable"
    
    print(f"{grado:<6} {mse_tr:<10.4f} {mse_te:<10.4f} {gap:<8.4f} {status:<15}")

print(f"\nConclusiones clave:")
print(f"  • Grado óptimo: {mejor_grado} (menor MSE en test)")
print(f"  • Overfitting: MSE train ↓, MSE test ↑")
print(f"  • Gap grande indica sobreajuste")
print(f"  • Validación en test es crucial")

## 5. 🎨 Técnicas Avanzadas de Feature Engineering

### 5.1 Features de Interacción

In [ ]:
# Demostración con múltiples variables y términos de interacción
def crear_features_interaccion(X):
    """
    Crea features de interacción para dos variables
    
    Para X = [x1, x2], crea:
    [x1, x2, x1², x1*x2, x2²]
    """
    if X.shape[1] != 2:
        raise ValueError("Esta función requiere exactamente 2 features")
    
    x1 = X[:, 0]
    x2 = X[:, 1]
    
    # Features originales + interacciones + polinomiales
    X_extended = np.column_stack([
        x1,           # x1
        x2,           # x2
        x1**2,        # x1²
        x1*x2,        # x1*x2 (interacción)
        x2**2         # x2²
    ])
    
    feature_names = ['x1', 'x2', 'x1²', 'x1*x2', 'x2²']
    
    return X_extended, feature_names

# Crear dataset sintético 2D con interacciones
np.random.seed(1)
n_samples = 50

# Variables independientes
x1 = np.random.uniform(-2, 2, n_samples)
x2 = np.random.uniform(-2, 2, n_samples)
X_base = np.column_stack([x1, x2])

# Variable dependiente con interacción real
y_interaccion = 3 + 2*x1 - x2 + 0.5*x1**2 + 1.5*x1*x2 - 0.3*x2**2 + np.random.normal(0, 0.5, n_samples)

print("Dataset con interacciones creado:")
print(f"  • Samples: {n_samples}")
print(f"  • Función real: y = 3 + 2x₁ - x₂ + 0.5x₁² + 1.5x₁x₂ - 0.3x₂²")
print(f"  • Término clave: 1.5x₁x₂ (interacción)")

# Comparar modelos con y sin interacciones
modelos_comparacion = {
    'Lineal Simple': X_base,
    'Con Interacciones': crear_features_interaccion(X_base)[0]
}

resultados_modelos = {}

for nombre_modelo, X_modelo in modelos_comparacion.items():
    # Normalizar features
    X_norm, mu, sigma = zscore_normalize_features(X_modelo)
    
    # Entrenar
    w_init = np.zeros(X_norm.shape[1])
    w, b = gradient_descent_simple(X_norm, y_interaccion, w_init, 0., 0.1, 1000)
    
    # Evaluar
    y_pred = X_norm @ w + b
    mse = np.mean((y_interaccion - y_pred)**2)
    r2 = 1 - np.sum((y_interaccion - y_pred)**2) / np.sum((y_interaccion - np.mean(y_interaccion))**2)
    
    resultados_modelos[nombre_modelo] = {
        'mse': mse,
        'r2': r2,
        'w': w,
        'b': b,
        'n_features': X_modelo.shape[1]
    }

# Mostrar comparación
print(f"\nComparación de modelos:")
print(f"{'Modelo':<20} {'Features':<9} {'MSE':<8} {'R²':<8}")
print("-" * 45)

for nombre, resultado in resultados_modelos.items():
    print(f"{nombre:<20} {resultado['n_features']:<9} {resultado['mse']:<8.3f} {resultado['r2']:<8.3f}")

# Análisis de importancia de features (modelo con interacciones)
X_inter, feature_names_inter = crear_features_interaccion(X_base)
w_inter = resultados_modelos['Con Interacciones']['w']

print(f"\nImportancia de features (modelo con interacciones):")
print(f"{'Feature':<8} {'Peso':<10} {'Importancia':<12}")
print("-" * 30)

for feature, peso in zip(feature_names_inter, w_inter):
    importancia = abs(peso)
    print(f"{feature:<8} {peso:<10.3f} {importancia:<12.3f}")

mejora_r2 = resultados_modelos['Con Interacciones']['r2'] - resultados_modelos['Lineal Simple']['r2']
print(f"\nMejora con interacciones: +{mejora_r2:.3f} en R²")

if abs(w_inter[3]) > 0.1:  # Peso de x1*x2
    print(f"✅ Término de interacción x1*x2 es significativo ({w_inter[3]:.3f})")
else:
    print(f"❌ Término de interacción x1*x2 no es significativo")

### 5.2 Visualización de Superficie de Decisión

In [ ]:
# Visualizar superficie de decisión 3D
def plot_superficie_3d(X, y, modelo_params, titulo):
    """
    Plotea superficie 3D del modelo
    """
    fig = plt.figure(figsize=(12, 5))
    
    # Subplot 1: Vista 3D
    ax1 = fig.add_subplot(121, projection='3d')
    
    # Puntos reales
    ax1.scatter(X[:, 0], X[:, 1], y, c='red', s=50, alpha=0.7)
    
    # Crear mesh para superficie
    x1_range = np.linspace(X[:, 0].min(), X[:, 0].max(), 20)
    x2_range = np.linspace(X[:, 1].min(), X[:, 1].max(), 20)
    X1_mesh, X2_mesh = np.meshgrid(x1_range, x2_range)
    
    # Crear features para el mesh
    mesh_points = np.column_stack([X1_mesh.ravel(), X2_mesh.ravel()])
    
    if 'con_interaccion' in titulo.lower():
        mesh_features, _ = crear_features_interaccion(mesh_points)
    else:
        mesh_features = mesh_points
    
    # Normalizar usando las mismas estadísticas
    mesh_norm = (mesh_features - modelo_params['mu']) / modelo_params['sigma']
    
    # Predicciones
    Z = mesh_norm @ modelo_params['w'] + modelo_params['b']
    Z = Z.reshape(X1_mesh.shape)
    
    # Superficie
    ax1.plot_surface(X1_mesh, X2_mesh, Z, alpha=0.6, cmap='viridis')
    
    ax1.set_xlabel('x1')
    ax1.set_ylabel('x2')
    ax1.set_zlabel('y')
    ax1.set_title(f'{titulo}\n3D View')
    
    # Subplot 2: Vista de contorno
    ax2 = fig.add_subplot(122)
    contour = ax2.contour(X1_mesh, X2_mesh, Z, levels=15, cmap='viridis', alpha=0.6)
    ax2.clabel(contour, inline=True, fontsize=8)
    
    # Puntos reales con color según valor y
    scatter = ax2.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='plasma', edgecolor='black')
    plt.colorbar(scatter, ax=ax2)
    
    ax2.set_xlabel('x1')
    ax2.set_ylabel('x2')
    ax2.set_title(f'{titulo}\nContour View')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Preparar parámetros para visualización
# Modelo lineal simple
X_lineal_norm, mu_lineal, sigma_lineal = zscore_normalize_features(X_base)
w_lineal, b_lineal = gradient_descent_simple(X_lineal_norm, y_interaccion, 
                                           np.zeros(2), 0., 0.1, 1000)

params_lineal = {
    'w': w_lineal,
    'b': b_lineal,
    'mu': mu_lineal,
    'sigma': sigma_lineal
}

# Modelo con interacciones
X_inter_full, _ = crear_features_interaccion(X_base)
X_inter_norm, mu_inter, sigma_inter = zscore_normalize_features(X_inter_full)
w_inter_full, b_inter_full = gradient_descent_simple(X_inter_norm, y_interaccion,
                                                    np.zeros(5), 0., 0.1, 1000)

params_inter = {
    'w': w_inter_full,
    'b': b_inter_full,
    'mu': mu_inter,
    'sigma': sigma_inter
}

# Visualizar ambos modelos
print("Superficies de decisión:")
print("=" * 25)
plot_superficie_3d(X_base, y_interaccion, params_lineal, "Modelo Lineal Simple")
plot_superficie_3d(X_base, y_interaccion, params_inter, "Modelo con Interacciones")

print("Diferencias observadas:")
print("  • Modelo lineal: Superficie plana")
print("  • Modelo con interacciones: Superficie curva")
print("  • Interacciones capturan relaciones más complejas")
print("  • Mejor ajuste a los datos reales")

## 6. 🧪 Ejercicios Prácticos

### Ejercicio 1: Selección Automática de Grado

In [ ]:
# Ejercicio 1: Implementar selección automática de grado polinomial
print("🧪 EJERCICIO 1: Selección Automática de Grado Polinomial")
print("=" * 60)

def seleccion_automatica_grado(x, y, grado_max=10, criterio='aic'):
    """
    Selecciona automáticamente el mejor grado polinomial usando criterios estadísticos
    
    Args:
        x, y: Datos
        grado_max: Grado máximo a probar
        criterio: 'aic', 'bic', o 'cv' (cross-validation simplificado)
    
    Returns:
        mejor_grado: Grado óptimo según el criterio
        resultados: Información detallada de todos los grados
    """
    n = len(y)
    resultados = []
    
    for grado in range(1, grado_max + 1):
        try:
            # Ajustar modelo
            w, b, mu, sigma = ajustar_polinomio(x, y, grado)
            y_pred = predecir_polinomio(x, w, b, mu, sigma, grado)
            
            # Métricas básicas
            mse = np.mean((y - y_pred)**2)
            r2 = 1 - np.sum((y - y_pred)**2) / np.sum((y - np.mean(y))**2)
            
            # Número de parámetros (incluyendo bias)
            k = grado + 1
            
            # Criterios de información
            if mse > 0:
                log_likelihood = -n/2 * np.log(2 * np.pi * mse) - n/2
                aic = -2 * log_likelihood + 2 * k
                bic = -2 * log_likelihood + k * np.log(n)
            else:
                aic = float('inf')
                bic = float('inf')
            
            # Cross-validation simplificado (Leave-One-Out aproximado)
            if criterio == 'cv' and n > 5:
                cv_errors = []
                indices = np.arange(n)
                step = max(1, n // 5)  # 5-fold aproximado
                
                for i in range(0, n, step):
                    # Train set (sin algunos puntos)
                    mask = np.ones(n, dtype=bool)
                    mask[i:i+step] = False
                    
                    if np.sum(mask) > grado:  # Suficientes puntos para entrenar
                        x_train_cv = x[mask]
                        y_train_cv = y[mask]
                        x_val_cv = x[~mask]
                        y_val_cv = y[~mask]
                        
                        # Entrenar en subset
                        w_cv, b_cv, mu_cv, sigma_cv = ajustar_polinomio(x_train_cv, y_train_cv, grado)
                        y_pred_cv = predecir_polinomio(x_val_cv, w_cv, b_cv, mu_cv, sigma_cv, grado)
                        
                        cv_errors.append(np.mean((y_val_cv - y_pred_cv)**2))
                
                cv_score = np.mean(cv_errors) if cv_errors else float('inf')
            else:
                cv_score = mse  # Fallback
            
            resultados.append({
                'grado': grado,
                'mse': mse,
                'r2': r2,
                'aic': aic,
                'bic': bic,
                'cv_score': cv_score,
                'n_params': k
            })
            
        except Exception as e:
            print(f"Error en grado {grado}: {e}")
    
    # Seleccionar mejor grado según criterio
    if criterio == 'aic':
        mejor_idx = np.argmin([r['aic'] for r in resultados])
    elif criterio == 'bic':
        mejor_idx = np.argmin([r['bic'] for r in resultados])
    elif criterio == 'cv':
        mejor_idx = np.argmin([r['cv_score'] for r in resultados])
    else:
        mejor_idx = np.argmax([r['r2'] for r in resultados])  # Fallback a R²
    
    mejor_grado = resultados[mejor_idx]['grado']
    
    return mejor_grado, resultados

# Probar con el dataset cuadrático
criterios = ['aic', 'bic', 'cv']
resultados_criterios = {}

for criterio in criterios:
    mejor_grado, detalles = seleccion_automatica_grado(x_cuad, y_cuad, 
                                                      grado_max=8, criterio=criterio)
    resultados_criterios[criterio] = {
        'mejor_grado': mejor_grado,
        'detalles': detalles
    }

# Mostrar resultados
print(f"Resultados de selección automática:")
print(f"{'Criterio':<10} {'Mejor Grado':<12} {'Interpretación':<20}")
print("-" * 45)

for criterio, resultado in resultados_criterios.items():
    grado = resultado['mejor_grado']
    
    if criterio == 'aic':
        interp = "Balance sesgo-varianza"
    elif criterio == 'bic':
        interp = "Penaliza complejidad"
    else:
        interp = "Rendimiento predictivo"
    
    print(f"{criterio.upper():<10} {grado:<12} {interp:<20}")

# Visualizar evolución de criterios
plt.figure(figsize=(15, 5))

detalles_aic = resultados_criterios['aic']['detalles']

grados = [d['grado'] for d in detalles_aic]
aics = [d['aic'] for d in detalles_aic]
bics = [d['bic'] for d in detalles_aic]
cvs = [d['cv_score'] for d in detalles_aic]

plt.subplot(1, 3, 1)
plt.plot(grados, aics, 'o-', linewidth=2, markersize=6)
plt.title('AIC vs Grado')
plt.xlabel('Grado Polinomial')
plt.ylabel('AIC (menor es mejor)')
plt.grid(True, alpha=0.3)
plt.axvline(x=resultados_criterios['aic']['mejor_grado'], color='red', linestyle='--', alpha=0.7)

plt.subplot(1, 3, 2)
plt.plot(grados, bics, 'o-', color='green', linewidth=2, markersize=6)
plt.title('BIC vs Grado')
plt.xlabel('Grado Polinomial')
plt.ylabel('BIC (menor es mejor)')
plt.grid(True, alpha=0.3)
plt.axvline(x=resultados_criterios['bic']['mejor_grado'], color='red', linestyle='--', alpha=0.7)

plt.subplot(1, 3, 3)
plt.plot(grados, cvs, 'o-', color='purple', linewidth=2, markersize=6)
plt.title('CV Score vs Grado')
plt.xlabel('Grado Polinomial')
plt.ylabel('CV MSE (menor es mejor)')
plt.grid(True, alpha=0.3)
plt.axvline(x=resultados_criterios['cv']['mejor_grado'], color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

print(f"\nConclusiones:")
print(f"  • AIC: Balance entre ajuste y complejidad")
print(f"  • BIC: Penaliza más la complejidad (grados menores)")
print(f"  • CV: Enfoque en capacidad predictiva real")
print(f"  • Para datos cuadráticos: Todos sugieren grados 2-3")

### Ejercicio 2: Features Personalizadas

In [ ]:
# Ejercicio 2: Crear y probar features personalizadas
print("🧪 EJERCICIO 2: Features Personalizadas y Transformaciones")
print("=" * 55)

def crear_features_personalizadas(x, tipo='completo'):
    """
    Crea diferentes tipos de features engineered
    
    Args:
        x: Feature original
        tipo: 'polinomial', 'trigonometrico', 'logaritmico', 'completo'
    """
    x = np.array(x).flatten()
    
    if tipo == 'polinomial':
        # Solo features polinomiales
        features = np.column_stack([
            x,           # x¹
            x**2,        # x²
            x**3,        # x³
            x**0.5       # √x (para x positivos)
        ])
        nombres = ['x', 'x²', 'x³', '√x']
        
    elif tipo == 'trigonometrico':
        # Features trigonométricas
        features = np.column_stack([
            x,
            np.sin(x),
            np.cos(x),
            np.sin(2*x),
            np.cos(2*x)
        ])
        nombres = ['x', 'sin(x)', 'cos(x)', 'sin(2x)', 'cos(2x)']
        
    elif tipo == 'logaritmico':
        # Features logarítmicas (para x positivos)
        x_pos = np.maximum(x, 0.01)  # Evitar log(0)
        features = np.column_stack([
            x,
            np.log(x_pos),
            np.log(x_pos + 1),
            x_pos * np.log(x_pos)
        ])
        nombres = ['x', 'log(x)', 'log(x+1)', 'x·log(x)']
        
    elif tipo == 'completo':
        # Combinación de diferentes tipos
        x_pos = np.maximum(x, 0.01)
        features = np.column_stack([
            x,                    # Lineal
            x**2,                 # Cuadrático
            x**3,                 # Cúbico
            np.sqrt(x_pos),       # Raíz
            np.sin(x),            # Seno
            np.cos(x),            # Coseno
            np.log(x_pos + 1),    # Logarítmico
            x * np.sin(x)         # Interacción
        ])
        nombres = ['x', 'x²', 'x³', '√x', 'sin(x)', 'cos(x)', 'log(x+1)', 'x·sin(x)']
    
    return features, nombres

# Probar con el dataset sinusoidal (más interesante para features trigonométricas)
x_sinus, y_sinus = datasets[2]

tipos_features = ['polinomial', 'trigonometrico', 'logaritmico', 'completo']
resultados_features = {}

print(f"Probando diferentes tipos de features en datos sinusoidales...")
print(f"\n{'Tipo Features':<15} {'N Features':<10} {'R²':<8} {'MSE':<10} {'Interpretación':<15}")
print("-" * 70)

for tipo in tipos_features:
    try:
        # Crear features
        X_custom, nombres_custom = crear_features_personalizadas(x_sinus, tipo)
        
        # Normalizar
        X_norm, mu, sigma = zscore_normalize_features(X_custom)
        
        # Entrenar
        w_init = np.zeros(X_norm.shape[1])
        w, b = gradient_descent_simple(X_norm, y_sinus, w_init, 0., 0.1, 1000)
        
        # Evaluar
        y_pred = X_norm @ w + b
        mse = np.mean((y_sinus - y_pred)**2)
        r2 = 1 - np.sum((y_sinus - y_pred)**2) / np.sum((y_sinus - np.mean(y_sinus))**2)
        
        # Interpretación
        if r2 > 0.9:
            interp = "Excelente"
        elif r2 > 0.7:
            interp = "Bueno"
        elif r2 > 0.5:
            interp = "Regular"
        else:
            interp = "Pobre"
        
        resultados_features[tipo] = {
            'X': X_custom,
            'nombres': nombres_custom,
            'w': w,
            'b': b,
            'mu': mu,
            'sigma': sigma,
            'r2': r2,
            'mse': mse
        }
        
        print(f"{tipo:<15} {X_custom.shape[1]:<10} {r2:<8.3f} {mse:<10.3f} {interp:<15}")
        
    except Exception as e:
        print(f"{tipo:<15} ERROR: {str(e)[:30]}")

# Visualizar el mejor modelo
mejor_tipo = max(resultados_features.keys(), key=lambda k: resultados_features[k]['r2'])
mejor_resultado = resultados_features[mejor_tipo]

print(f"\nMejor enfoque: {mejor_tipo} (R² = {mejor_resultado['r2']:.3f})")

# Visualizar comparación
plt.figure(figsize=(16, 12))

for i, (tipo, resultado) in enumerate(resultados_features.items()):
    plt.subplot(2, 2, i+1)
    
    # Datos originales
    plt.scatter(x_sinus, y_sinus, alpha=0.7, s=30, color='red', label='Datos reales')
    
    # Predicciones del modelo
    x_plot = np.linspace(np.min(x_sinus), np.max(x_sinus), 100)
    X_plot_custom, _ = crear_features_personalizadas(x_plot, tipo)
    X_plot_norm = (X_plot_custom - resultado['mu']) / resultado['sigma']
    y_plot = X_plot_norm @ resultado['w'] + resultado['b']
    
    plt.plot(x_plot, y_plot, 'b-', linewidth=2, label='Modelo')
    
    plt.title(f'{tipo.title()}\nR² = {resultado["r2"]:.3f}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Análisis de importancia de features del mejor modelo
print(f"\nAnálisis del mejor modelo ({mejor_tipo}):")
print(f"{'Feature':<12} {'Peso':<10} {'Importancia':<12}")
print("-" * 35)

pesos = mejor_resultado['w']
nombres = mejor_resultado['nombres']
importancias = np.abs(pesos)

# Ordenar por importancia
indices_ordenados = np.argsort(importancias)[::-1]

for idx in indices_ordenados:
    print(f"{nombres[idx]:<12} {pesos[idx]:<10.3f} {importancias[idx]:<12.3f}")

print(f"\nConclusiones del ejercicio:")
print(f"  • Features trigonométricas son ideales para datos sinusoidales")
print(f"  • Features polinomiales funcionan bien para curvas suaves")
print(f"  • Combinar tipos puede mejorar el rendimiento")
print(f"  • La elección de features debe basarse en el dominio del problema")

## 📚 Resumen y Conceptos Clave

### ✅ Lo que has aprendido:

#### 1. **Feature Engineering Fundamentals**:
   - **Concepto**: Crear nuevas features a partir de las existentes
   - **Objetivo**: Capturar relaciones no lineales con modelos lineales
   - **Principio**: El modelo sigue siendo lineal en los parámetros w

#### 2. **Regresión Polinomial**:
   - **Transformación**: x → [x, x², x³, ..., xⁿ]
   - **Modelo**: y = w₁x + w₂x² + w₃x³ + ... + b
   - **Flexibilidad**: Puede aproximar funciones complejas

#### 3. **Selección de Grado**:
   - **Underfitting**: Grado muy bajo → Bias alto
   - **Overfitting**: Grado muy alto → Varianza alta  
   - **Grado óptimo**: Balance sesgo-varianza
   - **Criterios**: AIC, BIC, Cross-validation

#### 4. **Tipos de Features**:
   - **Polinomiales**: x², x³, √x
   - **Trigonométricas**: sin(x), cos(x)
   - **Logarítmicas**: log(x), log(x+1)
   - **Interacciones**: x₁ × x₂
   - **Personalizadas**: Según dominio específico

#### 5. **Evaluación y Validación**:
   - **Train/Test split**: Detectar overfitting
   - **Métricas**: R², MSE, criterios de información
   - **Visualización**: Curvas de aprendizaje, superficies 3D

### 🎯 Aplicaciones Prácticas:
- **Datos cuadráticos**: Mejor con grado 2-3
- **Datos sinusoidales**: Features trigonométricas ideales
- **Datos exponenciales**: Transformaciones logarítmicas
- **Interacciones**: Términos x₁ × x₂ para relaciones complejas

### 🚀 Técnicas Avanzadas Cubiertas:
- **Selección automática de grado** usando AIC/BIC
- **Features de interacción** para múltiples variables
- **Visualización 3D** de superficies de decisión
- **Cross-validation** para validación robusta

### 💡 Best Practices:
1. **Analizar los datos** antes de elegir features
2. **Empezar simple** y aumentar complejidad gradualmente
3. **Usar validación cruzada** para selección de modelo
4. **Monitorear overfitting** con train/test split
   5. **Normalizar features** antes de aplicar polinomios
   6. **Validar con datos de prueba** para detectar overfitting
   7. **Interpretar resultados** en el contexto del problema

### ⚠️ Cuidados Importantes:
- **Overfitting**: Grados muy altos memorizan ruido
- **Underfitting**: Grados muy bajos no capturan patrones
- **Multicolinealidad**: Features polinomiales están correlacionadas
- **Escalabilidad**: Número de features crece exponencialmente
- **Interpretabilidad**: Modelos complejos son difíciles de explicar

### 🔮 Cuándo Usar Cada Técnica:

#### **Regresión Polinomial**:
- ✅ Datos con curvatura clara
- ✅ Relaciones suaves y continuas  
- ✅ Dataset pequeño a mediano
- ❌ Datos muy ruidosos
- ❌ Extrapolación fuera del rango

#### **Features Trigonométricas**:
- ✅ Patrones cíclicos o periódicos
- ✅ Datos de series temporales
- ✅ Fenómenos oscilatorios
- ❌ Relaciones estrictamente monótonas

#### **Features Logarítmicas**:
- ✅ Crecimiento exponencial
- ✅ Datos con gran rango dinámico
- ✅ Relaciones multiplicativas
- ❌ Datos con valores negativos o cero

#### **Features de Interacción**:
- ✅ Variables que se influencian mutuamente
- ✅ Efectos combinados evidentes
- ✅ Análisis de sensibilidad
- ❌ Muchas variables (explosión combinatoria)

### 🎯 Selección de Grado Polinomial:

| **Criterio** | **Descripción** | **Cuándo Usar** |
|--------------|-----------------|------------------|
| **R²** | Bondad de ajuste básico | Exploración inicial |
| **AIC** | Penaliza complejidad suavemente | Balance general |
| **BIC** | Penaliza complejidad fuertemente | Modelos parsimoniosos |
| **Cross-validation** | Estimación real de generalización | Selección final |
| **Hold-out test** | Validación independiente | Evaluación definitiva |

### 📊 Diagnóstico de Problemas:

#### **Síntomas de Underfitting**:
- R² bajo en entrenamiento Y prueba
- Residuos con patrones sistemáticos  
- Curva de predicción muy simple
- **Solución**: Aumentar complejidad del modelo

#### **Síntomas de Overfitting**:
- R² alto en entrenamiento, bajo en prueba
- Gap grande entre train/test performance
- Curva de predicción muy ondulada
- **Solución**: Reducir complejidad o regularizar

#### **Síntomas de Buen Ajuste**:
- R² similar en entrenamiento y prueba
- Residuos aleatorios sin patrones
- Curva suave que captura tendencias
- **Acción**: ¡Modelo listo para uso!

### 🔬 Workflow Recomendado:

```python
# 1. ANÁLISIS EXPLORATORIO
plt.scatter(x, y)  # Visualizar relación
# ¿Lineal? ¿Cuadrática? ¿Sinusoidal?

# 2. DIVISIÓN DE DATOS
X_train, X_test, y_train, y_test = train_test_split(...)

# 3. FEATURE ENGINEERING
X_poly = PolynomialFeatures(degree=2)
X_train_poly = X_poly.fit_transform(X_train)

# 4. NORMALIZACIÓN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)

# 5. ENTRENAMIENTO
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# 6. VALIDACIÓN
cv_scores = cross_val_score(model, X_train_scaled, y_train)

# 7. EVALUACIÓN FINAL
X_test_poly = X_poly.transform(X_test)
X_test_scaled = scaler.transform(X_test_poly)
predictions = model.predict(X_test_scaled)
```

### 📈 Métricas de Evaluación Importantes:

#### **Para Regresión**:
- **R²**: Proporción de varianza explicada (0-1, mayor mejor)
- **MSE**: Error cuadrático medio (≥0, menor mejor)
- **RMSE**: Raíz del MSE, en mismas unidades que y
- **MAE**: Error absoluto medio, robusto a outliers

#### **Para Selección de Modelo**:
- **AIC/BIC**: Criterios de información (menor mejor)
- **Cross-validation**: Estimación robusta de generalización
- **Learning curves**: Detectar overfitting/underfitting
- **Validation curves**: Optimizar hiperparámetros

### 🚨 Errores Comunes a Evitar:

1. **Data Leakage**: Normalizar antes de dividir train/test
   ```python
   # ❌ INCORRECTO
   X_scaled = scaler.fit_transform(X)  # Usa info de test
   X_train, X_test = train_test_split(X_scaled)
   
   # ✅ CORRECTO  
   X_train, X_test = train_test_split(X)
   X_train_scaled = scaler.fit_transform(X_train)
   X_test_scaled = scaler.transform(X_test)
   ```

2. **Extrapolación Peligrosa**: Polinomios se comportan mal fuera del rango
   ```python
   # Verificar rango de predicciones
   print(f"Train range: [{X_train.min()}, {X_train.max()}]")
   print(f"Test range: [{X_test.min()}, {X_test.max()}]")
   ```

3. **Ignorar Multicolinealidad**: Features polinomiales están correlacionadas
   ```python
   # Verificar correlación entre features
   correlation_matrix = np.corrcoef(X_poly.T)
   # Considerar Ridge/Lasso si hay multicolinealidad alta
   ```

4. **Optimizar en Test Set**: Solo usar test para evaluación final
   ```python
   # ❌ INCORRECTO: Optimizar grado usando test
   best_degree = tune_degree_on_test()  # Sesgo optimista
   
   # ✅ CORRECTO: Usar validation set o CV
   best_degree = cross_val_tune_degree()
   final_score = evaluate_on_test()  # Una sola vez
   ```

### 🎓 Extensiones Avanzadas:

#### **Regularización con Features Polinomiales**:
```python
from sklearn.linear_model import Ridge, Lasso

# Ridge: Penaliza magnitud de coeficientes
ridge = Ridge(alpha=1.0)

# Lasso: Puede eliminar features (selección automática)
lasso = Lasso(alpha=1.0)
```

#### **Splines para Flexibilidad Local**:
```python
from scipy.interpolate import UnivariateSpline

# Splines: Polinomios por pedazos
spline = UnivariateSpline(x, y, s=100)  # s controla suavidad
```

#### **Features Personalizadas Avanzadas**:
```python
# Features basadas en dominio específico
def create_domain_features(X):
    return np.column_stack([
        X,                    # Original
        np.exp(-X**2),        # Gaussiana
        1 / (1 + X**2),       # Cauchy
        np.tanh(X),           # Tangente hiperbólica
        np.sign(X) * X**2     # Cuadrática con signo
    ])
```

### 🔬 Casos de Estudio Típicos:

#### **Caso 1: Precio de Casas**
- **Features**: Tamaño, habitaciones, edad
- **Engineering**: Tamaño², habitaciones×pisos, log(edad+1)
- **Justificación**: Efectos no lineales y de interacción conocidos

#### **Caso 2: Ventas Estacionales**
- **Features**: Mes del año
- **Engineering**: sin(2πt/12), cos(2πt/12) para capturar estacionalidad
- **Justificación**: Patrones cíclicos evidentes

#### **Caso 3: Crecimiento Biológico**
- **Features**: Tiempo
- **Engineering**: t, t², log(t+1), exp(-t) para diferentes fases
- **Justificación**: Crecimiento exponencial inicial, saturación posterior

### 🌟 Próximos Pasos Sugeridos:

Después de dominar feature engineering básico:

1. **Regularización**: Ridge, Lasso, Elastic Net
2. **Selección de Features**: Métodos automáticos
3. **Algoritmos No Lineales**: Random Forest, SVM, Neural Networks
4. **Feature Learning**: Autoencoders, PCA, t-SNE
5. **Domain-Specific Engineering**: Según tu área de aplicación

### 🏆 Conclusión Final:

**Feature Engineering es tanto arte como ciencia:**
- 🧠 **Ciencia**: Métodos sistemáticos, validación estadística
- 🎨 **Arte**: Intuición del dominio, creatividad en transformaciones
- 🔧 **Práctica**: Experimentación iterativa, evaluación rigurosa

**Has aprendido a:**
- Transformar features para capturar patrones no lineales
- Evaluar y seleccionar grados polinomiales apropiados
- Crear features personalizadas según el problema
- Detectar y evitar overfitting
- Usar herramientas de validación robustas

**¡Ahora puedes hacer que los modelos lineales capturen relaciones complejas!** 🚀

---

*En el próximo notebook veremos cómo **Scikit-Learn** automatiza y optimiza todos estos procesos para uso profesional.*